## GPT 기반 2023 수능 국어 문제 풀기

### 프로그램 개요
이 프로그램은 OpenAI의 GPT 모델을 활용하여 대학수학능력시험(수능) 문제를 자동으로 풀이하고 모델과 프롬프트의 성능을 평가하는 시스템입니다. 다양한 프롬프트 방법론을 적용하여 모델의 문제 해결 능력을 체계적으로 분석하고 비교할 수 있습니다.

라이브러리 임포트 및 설정

In [2]:
# 필요한 라이브러리 import
import requests          # GitHub에서 데이터 다운로드용
import openai           # OpenAI API 사용
from openai import OpenAI
from typing import Dict, List, Tuple  # 타입 힌팅
import time             # 실행 시간 측정 및 API 호출 간격 조절
import re               # 정규표현식 (답안 파싱용)
import pandas as pd     # 데이터 처리 및 결과 저장
import tiktoken         # OpenAI 토큰 계산
import random           # API 실패 시 랜덤 답안 생성
from dataclasses import dataclass  # 결과 데이터 구조화
from dotenv import load_dotenv     # 환경변수 로드
import os               # 파일 시스템 작업
from pathlib import Path           # 경로 처리
import json             # JSON 파일 처리

## 📋 테스트 결과 데이터 구조 정의

- @dataclass 데코레이터를 사용하여 테스트 결과를 체계적으로 저장
- AI 모델별, 프롬프트 전략별 성능 지표를 구조화된 형태로 관리
- 정답률, 실행시간, 상세 결과 등 다양한 메트릭을 포함
- 타입 힌트를 통해 각 필드의 데이터 타입을 명시하여 코드 가독성 향상

In [3]:
# OpenAI API 설정
load_dotenv()  # .env 파일에서 환경변수 로드
openai.api_key = os.getenv("OPENAI_API_KEY")  # API 키 설정

@dataclass
class TestResult:
    """테스트 결과를 저장하는 데이터 클래스"""
    model: str              # 사용된 AI 모델명 (예: gpt-4o, gpt-4o-mini)
    prompt_method: str      # 사용된 프롬프트 전략명
    correct_answers: int    # 맞힌 문제 개수
    total_questions: int    # 전체 문제 개수
    score: float           # 정답률 (백분율)
    total_score: float     # 획득한 총 점수
    execution_time: float  # 전체 실행 시간 (초)
    detailed_results: pd.DataFrame  # 문제별 상세 결과 데이터

In [4]:
# 프롬프트 템플릿
PROMPTS = {
    # 기본적인 zero-shot 프롬프트 (추가 설명 없이 문제만 제시)
    "zero_shot": """다음 지문을 읽고 문제를 풀어주세요.
    
    지문:
    {paragraph}
    
    문제: {question}
    {question_plus}
    
    선택지:
    {choices1}
    {choices2}
    {choices3}
    {choices4}
    {choices5}
    
    문제를 풀어줘. 정답만 번호로 줘

    ###답변 형태 ###
    정답: """,
    
    # 감정적 호소를 통한 동기부여 프롬프트
    "emotional_appeal": """🎯 인생에서 최고로 중요한 수능 국어 문제입니다.
    당신이 소중하게 생각하는 할머니께서 당신이 좋은 성적을 얻기만을 기다리고 계십니다. 당신의 실력을 보여주세요.
    이 문제는 많은 수험생들이 어려워하는 유형입니다. 하지만 당신은 분명히 풀 수 있습니다! 
    차근차근 읽고, 단계별로 생각해서 최선의 답을 찾아주세요.
    자신감을 가지고 열정적으로 풀어보세요.
    
    지문:
    {paragraph}
    
    ❓ 문제: {question}
    
    {question_plus}
    
    선택지를 신중하게 검토해보세요:
    {choices1}
    {choices2}
    {choices3}
    {choices4}
    {choices5}
    
    깊이 생각해보고, 지문의 내용을 정확히 파악한 후 답을 선택하세요.
    당신은 반드시 정답을 찾을 수 있습니다.
    
    이 문제를 틀리면 정말 좋은 기회를 놓치게 될 것입니다. 신중하게, 그리고 확신을 가지고 답해주세요.
    
    ###답변 형태 ###
    정답: """,

    # 전문가 역할 부여 프롬프트 (체계적 분석 유도)
    "expert_role": """
    당신은 국어교육 전문가입니다. 다음 지문과 문제를 분석하여 정확한 답을 제시해주세요.
    
    지문:
    {paragraph}
    
    문제: {question}
    {question_plus}
    
    선택지:
    {choices1}
    {choices2}
    {choices3}
    {choices4}
    {choices5}
    
    ###답변 형태 ###
    정답: """
    }

print("데이터 클래스 및 프롬프트 정의 완료!")



데이터 클래스 및 프롬프트 정의 완료!


## 🧠 한국어 수능 문제 해결 클래스 정의

GitHub에서 2023년 수능 국어 데이터를 자동으로 로드하는 기능 구현
OpenAI API 클라이언트와 토큰 계산기(tiktoken)를 초기화하여 AI 모델 활용 준비
개별 문제 단위와 지문별 그룹화 두 가지 데이터 구조로 유연한 처리 지원
원본 JSON 데이터를 파싱하여 문제별 정보(지문, 선택지, 정답, 배점 등)를 구조화
예외 처리를 통해 네트워크 오류나 데이터 로드 실패 상황에 대응

---
### 💡 추가 정보
>tiktoken은 OpenAI의 토큰 계산 라이브러리로, API 비용 관리와 토큰 제한 확인에 필수적임. requests.raise_for_status()는 HTTP 오류 상태 코드에 대해 예외를 발생시켜 안전한 API 호출을 보장함.

In [5]:
class KoreanSuneungSolver:
    def __init__(self):
        self.data = None                # 개별 문제 단위로 저장된 데이터
        self.grouped_data = None        # 지문별로 그룹화된 데이터 (효율적 처리용)
        self.results = {}              # 테스트 결과를 저장하는 딕셔너리
        self.client = OpenAI()         # OpenAI API 클라이언트
        self.tokenizer = tiktoken.encoding_for_model("gpt-4o")  # 토큰 계산용
        
    def load_data(self):
        """GitHub에서 2023년 수능 국어 데이터 로드 및 지문별 그룹화"""
        github_url = "https://raw.githubusercontent.com/NomaDamas/KICE_slayer_AI_Korean/master/data/2023_11_KICE.json"
        try:
            response = requests.get(github_url, timeout=30)
            response.raise_for_status()
            raw_data = response.json()
            
            # 기존 방식으로 변환 (호환성 유지)
            total_result = []
            for data in raw_data:
                paragraph_id = data['id']
                paragraph = data['paragraph']
                data_type = data['type']
                problems_lst = data['problems']
                
                for problem in problems_lst:
                    question = problem['question']
                    choices = [f"{i+1}번 {choice}" for i, choice in enumerate(problem['choices'])]
                    answer = problem['answer']
                    score = problem['score']
                    question_plus = problem.get('question_plus', '')
                    
                    problem_dict = {
                        'paragraph_id': paragraph_id,
                        'type': data_type,
                        'paragraph': paragraph,
                        'question': question,
                        'question_plus': question_plus,
                        'choices': choices,
                        'answer': answer,
                        'score': score,
                    }
                    total_result.append(problem_dict)
            
            self.data = total_result
            
            # 지문별로 그룹화 (멘토 스타일)
            self.grouped_data = self._group_by_paragraph(raw_data)
            
            print(f"데이터 로드 완료: {len(self.data)} 문제, {len(self.grouped_data)} 지문")
            return True
        except Exception as e:
            print(f"데이터 로드 실패: {e}")
            return False
    
    def _group_by_paragraph(self, raw_data):
        """원본 데이터를 지문별로 그룹화"""
        grouped = []
        for data in raw_data:
            paragraph_data = {
                'id': data['id'],
                'type': data['type'],
                'paragraph': data['paragraph'],
                'problems': []
            }
            
            for problem in data['problems']:
                problem_dict = {
                    'question': problem['question'],
                    'question_plus': problem.get('question_plus', ''),
                    'choices': problem['choices'],  # 원본 형태 유지
                    'answer': problem['answer'],
                    'score': problem['score']
                }
                paragraph_data['problems'].append(problem_dict)
            
            grouped.append(paragraph_data)
        
        return grouped
    
# 인스턴스 생성 및 데이터 로드
solver = KoreanSuneungSolver()
print("KoreanSuneungSolver 클래스 정의 완료!")

KoreanSuneungSolver 클래스 정의 완료!


## 🛠️ 핵심 유틸리티 메서드 구현

- 답안 정규화: AI 모델의 다양한 출력 형태를 1-5번 정수로 변환하는 로직 구현
- 선택지 형태를 자동 감지하여 "1번, 2번..." 형식으로 통일화
- 예외 처리와 함께 안전한 API 호출 및 랜덤 답안 fallback 제공

💡 추가 정보: temperature=0은 AI 모델의 창의성을 최소화하여 일관된 답변을 얻는 설정. 동적 메서드 바인딩을 통해 클래스 외부에서 정의한 함수를 클래스 메서드로 추가할 수 있습니다.

In [6]:
# KoreanSuneungSolver 클래스에 유틸리티 메서드 추가
def normalize_answer(self, text: str) -> int:
    """답안 정규화"""
    if hasattr(text, 'content'):
        result = text.content
    else:
        result = str(text)
        
    result = result.split("정답:")[-1]
    for num in range(1, 6):
        if str(num) in result:
            return num
        
    return random.randint(1, 5)
    
def prediction(self, prompt: str, model: str = "gpt-4o"):
    """OpenAI API 호출"""
    try:
        completion = self.client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=100,
            temperature=0
        )
        return completion.choices[0].message
    except Exception as e:
        print(f"API 호출 실패: {e}")
        return None
    
def build_prompt_for_problem(self, prompt_name: str, paragraph: str, problem: dict) -> str:
    """개별 문제에 대한 프롬프트 생성"""
    if prompt_name not in PROMPTS:
        raise ValueError(f"지원하지 않는 프롬프트: {prompt_name}")
    
    prompt_template = PROMPTS[prompt_name]
    question_plus_text = f"추가 정보: {problem['question_plus']}" if problem['question_plus'] else ""
        
    # 선택지 포맷팅 (멘토 스타일에 맞춰 조정)
    choices = problem['choices']
    if isinstance(choices[0], str) and not choices[0].startswith('1번'):
        # 원본 형태라면 번호 추가
        formatted_choices = [f"{i+1}번 {choice}" for i, choice in enumerate(choices)]
    else:
        # 이미 번호가 있다면 그대로 사용
        formatted_choices = choices
        
    prompt = prompt_template.format(
        paragraph=paragraph,
        question=problem['question'],
        question_plus=question_plus_text,
        choices1=formatted_choices[0],
        choices2=formatted_choices[1],
        choices3=formatted_choices[2],
        choices4=formatted_choices[3],
        choices5=formatted_choices[4],
    )
        
    return prompt

# 메서드를 클래스에 바인딩
KoreanSuneungSolver.normalize_answer = normalize_answer
KoreanSuneungSolver.prediction = prediction
KoreanSuneungSolver.build_prompt_for_problem = build_prompt_for_problem
    
print("유틸리티 메서드 추가 완료!")


유틸리티 메서드 추가 완료!


## 📝 지문별 문제 해결 엔진

- 실시간 진행상황 표시: 각 지문과 문제별로 ID, 길이, 문제 수를 콘솔에 출력하여 진행상황 모니터링
- 문제별 성능 측정: 개별 문제의 처리 시간을 측정하고 정답 여부를 즉시 확인
- 상세 결과 수집: 예측값, 정답, 획득점수, AI 응답 텍스트 등을 구조화하여 저장
- API 호출 제한 대응: 0.5초 간격으로 API 호출하여 OpenAI 레이트 리밋 준수
- 실패 안전장치: API 호출 실패 시 랜덤 답안으로 fallback하여 테스트 연속성 보장

--- 
> 💡 추가 정보: time.sleep(0.5)는 API 레이트 리밋을 피하기 위한 중요한 설정. 문자열 슬라이싱과 조건부 표현식을 사용하여 긴 텍스트를 안전하게 출력 형태로 변환합니다.

In [7]:
def solve_paragraph_problems(self, paragraph_data: dict, prompt_name: str, model: str) -> List[dict]:
    """한 지문의 모든 문제를 해결하고 실시간 결과 출력"""
    paragraph = paragraph_data['paragraph']
    problems = paragraph_data['problems']
    paragraph_id = paragraph_data['id']
        
    print(f"\n📖 지문 ID: {paragraph_id}")
    print(f"   지문 길이: {len(paragraph)}자, 문제 수: {len(problems)}")
    print("-" * 50)
        
    results = []
        
    for i, problem in enumerate(problems):
        problem_start_time = time.time()
            
        # 프롬프트 생성
        prompt = self.build_prompt_for_problem(prompt_name, paragraph, problem)
            
        # API 호출
        response = self.prediction(prompt, model)
        if response is None:
            pred = random.randint(1, 5)
            response_text = "API 호출 실패"
        else:
            pred = self.normalize_answer(response)
            response_text = response.content
            
        # 정답 확인
        correct = problem['answer']
        score = problem['score']
        is_correct = (pred == correct)
        earned_score = score if is_correct else 0
            
        problem_time = time.time() - problem_start_time
            
        # 실시간 결과 출력
        result_symbol = "✅" if is_correct else "❌"
        print(f"  {result_symbol} 문제 {i+1}: 예측={pred}, 정답={correct}, 점수={earned_score}/{score}")
        print(f"     응답: {response_text[:100]}{'...' if len(response_text) > 100 else ''}")
        print(f"     처리시간: {problem_time:.2f}초")
            
        # 결과 저장
        result_data = {
            'paragraph_id': paragraph_id,
            'problem_num': i + 1,
            'question': problem['question'][:100] + "..." if len(problem['question']) > 100 else problem['question'],
            'pred': pred,
            'answer': correct,
            'score': earned_score,
            'max_score': score,
            'is_correct': is_correct,
            'response': response_text,
            'processing_time': problem_time
        }
        results.append(result_data)
            
        # API 제한 고려
        time.sleep(0.5)
        
    return results
# 메서드를 클래스에 바인딩
KoreanSuneungSolver.solve_paragraph_problems = solve_paragraph_problems

print("문제 해결 메서드 추가 완료!")


문제 해결 메서드 추가 완료!


## 🧪 통합 테스트 실행 시스템

- 문제 수 제한 기능: max_questions 파라미터로 테스트 범위를 유연하게 조절 가능
- 진행상황 실시간 모니터링: 지문별 진행률과 누적 점수를 실시간으로 표시하여 테스트 현황 파악
- 자동 결과 저장: 테스트 결과를 CSV 파일로 자동 저장하여 후속 분석에 활용
- 종합 성능 지표 계산: 정답률, 총점, 실행시간 등 다양한 성능 메트릭을 자동 계산
- 구조화된 결과 반환: TestResult 데이터클래스로 결과를 체계적으로 관리하고 비교 분석 지원

In [8]:
def run_test(self, prompt_name: str, model: str = "gpt-4o", max_questions: int = None) -> TestResult:
        """지문별 그룹화된 테스트 실행"""
        if self.grouped_data is None:
            raise ValueError("데이터를 먼저 로드해주세요.")
        
        print(f"\n{'='*60}")
        print(f"🚀 테스트 시작: {model} + {prompt_name}")
        print(f"⏰ 시작 시간: {time.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"{'='*60}")
        
        start_time = time.time()
        all_results = []
        total_score = 0
        question_count = 0
        
        # 문제 수 제한 처리
        test_data = self.grouped_data
        if max_questions:
            # 문제 수를 제한하는 경우, 지문별로 누적하여 계산
            limited_data = []
            current_count = 0
            for paragraph_data in self.grouped_data:
                if current_count >= max_questions:
                    break
                remaining = max_questions - current_count
                if len(paragraph_data['problems']) <= remaining:
                    limited_data.append(paragraph_data)
                    current_count += len(paragraph_data['problems'])
                else:
                    # 지문의 일부 문제만 포함
                    truncated_data = paragraph_data.copy()
                    truncated_data['problems'] = paragraph_data['problems'][:remaining]
                    limited_data.append(truncated_data)
                    break
            test_data = limited_data
        
        # 각 지문별로 문제 해결
        for paragraph_idx, paragraph_data in enumerate(test_data):
            print(f"\n🔍 진행률: {paragraph_idx + 1}/{len(test_data)} 지문")
            
            paragraph_results = self.solve_paragraph_problems(paragraph_data, prompt_name, model)
            all_results.extend(paragraph_results)
            
            # 점수 누적
            for result in paragraph_results:
                total_score += result['score']
                question_count += 1
            
            print(f"   지문 완료 - 현재 총점: {total_score}점")
        
        end_time = time.time()
        execution_time = end_time - start_time
        
        # DataFrame 생성
        df = pd.DataFrame(all_results)
        
        # 통계 계산
        correct_answers = df['is_correct'].sum()
        total_questions = len(df)
        max_possible_score = df['max_score'].sum()
        percentage = (correct_answers / total_questions) * 100
        
        # CSV 저장
        os.makedirs("data/results", exist_ok=True)
        csv_path = f"data/results/{prompt_name}_{model}.csv"
        df.to_csv(csv_path, index=False, encoding='utf-8')
        
        result = TestResult(
            model=model,
            prompt_method=prompt_name,
            correct_answers=correct_answers,
            total_questions=total_questions,
            score=percentage,
            total_score=total_score,
            execution_time=execution_time,
            detailed_results=df
        )
        
        self.results[f"{prompt_name}_{model}"] = result
        
        print(f"\n{'='*60}")
        print(f"✅ 테스트 완료: {prompt_name} + {model}")
        print(f"⏰ 완료 시간: {time.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"📊 최종 결과:")
        print(f"   총점: {total_score}/{max_possible_score}점 ({total_score/max_possible_score*100:.1f}%)")
        print(f"   정답률: {percentage:.1f}% ({correct_answers}/{total_questions})")
        print(f"   소요시간: {execution_time:.2f}초")
        print(f"{'='*60}")
        
        return result
    
# 메서드를 클래스에 바인딩
KoreanSuneungSolver.run_test = run_test

print("테스트 실행 메서드 추가 완료!")

테스트 실행 메서드 추가 완료!


분석 및 출력 메서드(1/2)

In [9]:
def print_progress(self, current: int, total: int, desc: str = "진행중"):
        """진행률 출력 함수"""
        percentage = (current / total) * 100
        bar_length = 30
        filled_length = int(bar_length * current // total)
        bar = '█' * filled_length + '-' * (bar_length - filled_length)
        print(f'\r{desc}: |{bar}| {current}/{total} ({percentage:.1f}%)', end='', flush=True)
        if current == total:
            print()
    
def print_comparison_table(self):
        """모든 결과를 비교 테이블로 출력"""
        if not self.results:
            print("비교할 결과가 없습니다.")
            return
        
        print("\n" + "="*100)
        print("📊 종합 결과 비교 테이블")
        print("="*100)
        
        # 테이블 헤더
        print(f"{'모델':<15} {'프롬프트':<18} {'정답률':<8} {'총점':<8} {'정답수/총문제':<12} {'실행시간':<10}")
        print("-" * 100)
        
        # 결과 정렬 (정답률 기준 내림차순)
        sorted_results = sorted(self.results.items(), key=lambda x: x[1].score, reverse=True)
        
        for name, result in sorted_results:
            accuracy = f"{result.score:.1f}%"
            score_ratio = f"{result.total_score:.0f}"
            answer_ratio = f"{result.correct_answers}/{result.total_questions}"
            exec_time = f"{result.execution_time:.1f}초"
            
            print(f"{result.model:<15} {result.prompt_method:<18} {accuracy:<8} {score_ratio:<8} {answer_ratio:<12} {exec_time:<10}")
        
        print("-" * 100)
        
        # 최고 성능 강조
        if sorted_results:
            best_result = sorted_results[0][1]
            print(f"!! 최고 성능: {best_result.model} + {best_result.prompt_method}")
            print(f"   정답률: {best_result.score:.1f}% | 총점: {best_result.total_score:.0f}점 | 시간: {best_result.execution_time:.1f}초")
        
        # 프롬프트별 평균 성능
        self.print_prompt_analysis()
        
        # 모델별 평균 성능
        self.print_model_analysis()
    
def print_prompt_analysis(self):
        """프롬프트별 성능 분석"""
        print(f"\n📈 프롬프트별 평균 성능:")
        prompt_stats = {}
        
        for result in self.results.values():
            prompt = result.prompt_method
            if prompt not in prompt_stats:
                prompt_stats[prompt] = {'scores': [], 'times': []}
            prompt_stats[prompt]['scores'].append(result.score)
            prompt_stats[prompt]['times'].append(result.execution_time)
        
        for prompt, stats in prompt_stats.items():
            avg_score = sum(stats['scores']) / len(stats['scores'])
            avg_time = sum(stats['times']) / len(stats['times'])
            print(f"  {prompt:<18}: 평균 정답률 {avg_score:.1f}% | 평균 시간 {avg_time:.1f}초")
    
def print_model_analysis(self):
        """모델별 성능 분석"""
        print(f"\n🤖 모델별 평균 성능:")
        model_stats = {}
        
        for result in self.results.values():
            model = result.model
            if model not in model_stats:
                model_stats[model] = {'scores': [], 'times': []}
            model_stats[model]['scores'].append(result.score)
            model_stats[model]['times'].append(result.execution_time)
        
        for model, stats in model_stats.items():
            avg_score = sum(stats['scores']) / len(stats['scores'])
            avg_time = sum(stats['times']) / len(stats['times'])
            print(f"  {model:<15}: 평균 정답률 {avg_score:.1f}% | 평균 시간 {avg_time:.1f}초")
    
# 메서드들을 클래스에 바인딩
KoreanSuneungSolver.print_progress = print_progress
KoreanSuneungSolver.print_comparison_table = print_comparison_table
KoreanSuneungSolver.print_prompt_analysis = print_prompt_analysis
KoreanSuneungSolver.print_model_analysis = print_model_analysis

print("분석 및 출력 메서드들 (1/2) 추가 완료!")


분석 및 출력 메서드들 (1/2) 추가 완료!


## 🧪 분석 및 출력 메서드(2/2)

In [10]:
def show_sample_responses(self):
        """각 프롬프트 타입에 대한 정답과 오답의 예시"""
        print(f"\n📋 프롬프트별 응답 예시:")
        
        for i, (name, result) in enumerate(self.results.items()):
            df = result.detailed_results
            
            # 정답 예시
            correct_samples = df[df['is_correct'] == True]
            incorrect_samples = df[df['is_correct'] == False]
            
            if len(correct_samples) > 0 and len(incorrect_samples) > 0:
                print(f"\n{'='*60}")
                print(f"📝 {result.model} + {result.prompt_method}")
                print(f"{'='*60}")
                
                # 정답 예시
                true_sample = correct_samples.iloc[0]
                print(f"✅ 정답 예시:")
                print(f"  문제: {true_sample['question'][:100]}...")
                print(f"  예측: {true_sample['pred']} | 정답: {true_sample['answer']}")
                print(f"  응답: {true_sample['response'][:150]}...\n")
                
                # 오답 예시
                false_sample = incorrect_samples.iloc[0]
                print(f"❌ 오답 예시:")
                print(f"  문제: {false_sample['question'][:100]}...")
                print(f"  예측: {false_sample['pred']} | 정답: {false_sample['answer']}")
                print(f"  응답: {false_sample['response'][:150]}...\n")

def analyze_results(self):
        """결과 분석 및 출력 (기존 함수 개선)"""
        if not self.results:
            print("분석할 결과가 없습니다.")
            return
        
        # 비교 테이블 출력
        self.print_comparison_table()
        
        # 상세 분석
        print(f"\n🔍 상세 분석:")
        self.show_sample_responses()
    
def export_detailed_results(self):
        """상세 결과를 JSON과 CSV로 내보내기"""
        if not self.results:
            print("내보낼 결과가 없습니다.")
            return
        
        # JSON 형태로 요약 결과 저장
        summary_data = []
        for name, result in self.results.items():
            summary_data.append({
                'model': str(result.model),                  
                'prompt_method': str(result.prompt_method),
                'total_score': float(result.total_score),     
                'score_percentage': float(result.score),
                'correct_answers': int(result.correct_answers), 
                'total_questions': int(result.total_questions),
                'execution_time': float(result.execution_time)
            })
        
        os.makedirs("data/results", exist_ok=True)
        
        # 요약 결과 저장
        with open("data/results/comprehensive_summary.json", 'w', encoding='utf-8') as f:
            json.dump(summary_data, f, ensure_ascii=False, indent=2)
        
        # CSV 형태로도 요약 저장
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_csv("data/results/comprehensive_summary.csv", index=False, encoding='utf-8')
        
        print(f"\n💾 종합 결과 저장 완료:")
        print(f"   - JSON 요약: data/results/comprehensive_summary.json")
        print(f"   - CSV 요약: data/results/comprehensive_summary.csv")
        print(f"   - 개별 상세 결과: data/results/ 폴더의 각 CSV 파일들")
    
# 메서드들을 클래스에 바인딩
KoreanSuneungSolver.show_sample_responses = show_sample_responses
KoreanSuneungSolver.analyze_results = analyze_results
KoreanSuneungSolver.export_detailed_results = export_detailed_results

print("분석 및 출력 메서드들 (2/2) 추가 완료!")

분석 및 출력 메서드들 (2/2) 추가 완료!


종합 테스트 메서드

In [11]:
def run_comprehensive_test(self, models: List[str] = None, max_questions: int = None):
        """종합 테스트 실행 - 모든 프롬프트와 모델 조합 테스트"""
        if models is None:
            models = ["gpt-4o-mini", "gpt-4o"]
        
        # 3가지 프롬프트 기법 사용
        prompt_methods = ["zero_shot", "emotional_appeal", "expert_role"]
        
        print("=" * 80)
        print("🎯 2023 수능 국어 AI 풀이 프로그램 (종합 테스트)")
        print("=" * 80)
        print(f"📊 테스트할 문제 수: {max_questions if max_questions else '전체'}")
        print(f"🤖 모델: {', '.join(models)}")
        print(f"💡 프롬프트 기법: {', '.join(prompt_methods)}")
        print(f"🔥 총 테스트 조합: {len(models)} × {len(prompt_methods)} = {len(models) * len(prompt_methods)}개")
        print("=" * 80)
        
        total_combinations = len(models) * len(prompt_methods)
        current_combination = 0
        
        for model in models:
            for prompt_method in prompt_methods:
                current_combination += 1
                try:
                    print(f"\n🚀 테스트 조합 {current_combination}/{total_combinations}: {model} + {prompt_method}")
                    self.run_test(prompt_method, model, max_questions)
                    print(f"✅ 조합 {current_combination} 완료")
                    print("\n" + "-" * 60)
                except Exception as e:
                    print(f"❌ 테스트 실패 - {model} + {prompt_method}: {e}")
                    print("-" * 60)
        
        # 테스트 완료 후 결과 테이블 출력
        self.print_comparison_table()
    
# 메서드를 클래스에 바인딩
KoreanSuneungSolver.run_comprehensive_test = run_comprehensive_test

print("종합 테스트 메서드 추가 완료!")
print("모든 클래스 메서드가 정의되었습니다!")

종합 테스트 메서드 추가 완료!
모든 클래스 메서드가 정의되었습니다!


데이터 로드 및 확인

In [12]:
# 데이터 로드
if not solver.load_data():
    print("데이터 로드에 실패했습니다.")
else:
    print(f"✅ 데이터 로드 성공!")
    print(f"📊 총 문제 수: {len(solver.data)}")
    print(f"📖 총 지문 수: {len(solver.grouped_data)}")
    
    # 첫 번째 지문 미리보기
    if solver.grouped_data:
        first_paragraph = solver.grouped_data[0]
        print(f"\n🔍 첫 번째 지문 미리보기:")
        print(f"   ID: {first_paragraph['id']}")
        print(f"   타입: {first_paragraph['type']}")
        print(f"   지문 길이: {len(first_paragraph['paragraph'])}자")
        print(f"   문제 수: {len(first_paragraph['problems'])}")

데이터 로드 완료: 45 문제, 11 지문
✅ 데이터 로드 성공!
📊 총 문제 수: 45
📖 총 지문 수: 11

🔍 첫 번째 지문 미리보기:
   ID: 2023_11_KICE_1-3
   타입: 0
   지문 길이: 978자
   문제 수: 3


단일 테스트 실행 (테스트용)

In [13]:
# 단일 프롬프트와 모델로 소규모 테스트 실행
print("🧪 단일 테스트 실행 (5문제로 제한)")
print("=" * 50)

# 테스트 실행 (5문제만)
test_result = solver.run_test(
    prompt_name="zero_shot",
    model="gpt-4o-mini", 
    max_questions=5
)

print(f"\n📊 단일 테스트 결과:")
print(f"   모델: {test_result.model}")
print(f"   프롬프트: {test_result.prompt_method}")
print(f"   정답률: {test_result.score:.1f}%")
print(f"   총점: {test_result.total_score}점")
print(f"   정답 수: {test_result.correct_answers}/{test_result.total_questions}")

🧪 단일 테스트 실행 (5문제로 제한)

🚀 테스트 시작: gpt-4o-mini + zero_shot
⏰ 시작 시간: 2025-05-25 17:41:30

🔍 진행률: 1/2 지문

📖 지문 ID: 2023_11_KICE_1-3
   지문 길이: 978자, 문제 수: 3
--------------------------------------------------
  ✅ 문제 1: 예측=4, 정답=4, 점수=2/2
     응답: 정답: 4
     처리시간: 1.32초
  ❌ 문제 2: 예측=4, 정답=5, 점수=0/3
     응답: 정답: 4번
     처리시간: 0.94초
  ✅ 문제 3: 예측=1, 정답=1, 점수=2/2
     응답: 정답: 1번
     처리시간: 0.90초
   지문 완료 - 현재 총점: 4점

🔍 진행률: 2/2 지문

📖 지문 ID: 2023_11_KICE_4-9
   지문 길이: 2158자, 문제 수: 2
--------------------------------------------------
  ✅ 문제 1: 예측=4, 정답=4, 점수=2/2
     응답: 정답: 4번
     처리시간: 0.93초
  ✅ 문제 2: 예측=5, 정답=5, 점수=2/2
     응답: 정답: 5
     처리시간: 1.21초
   지문 완료 - 현재 총점: 8점

✅ 테스트 완료: zero_shot + gpt-4o-mini
⏰ 완료 시간: 2025-05-25 17:41:38
📊 최종 결과:
   총점: 8/11점 (72.7%)
   정답률: 80.0% (4/5)
   소요시간: 7.82초

📊 단일 테스트 결과:
   모델: gpt-4o-mini
   프롬프트: zero_shot
   정답률: 80.0%
   총점: 8점
   정답 수: 4/5


결과 분석 및 시각화

In [24]:
# 결과 분석
print("📈 결과 분석")
print("=" * 50)

solver.analyze_results()

📈 결과 분석

📊 종합 결과 비교 테이블
모델              프롬프트               정답률      총점       정답수/총문제      실행시간      
----------------------------------------------------------------------------------------------------
gpt-4o          zero_shot          95.6%    96       43/45        97.2초     
gpt-4o          emotional_appeal   84.4%    85       38/45        123.4초    
gpt-4o          expert_role        82.2%    83       37/45        122.2초    
gpt-4o-mini     zero_shot          71.1%    71       32/45        97.7초     
gpt-4o-mini     expert_role        57.8%    58       26/45        189.4초    
gpt-4o-mini     emotional_appeal   55.6%    56       25/45        140.1초    
----------------------------------------------------------------------------------------------------
!! 최고 성능: gpt-4o + zero_shot
   정답률: 95.6% | 총점: 96점 | 시간: 97.2초

📈 프롬프트별 평균 성능:
  zero_shot         : 평균 정답률 83.3% | 평균 시간 97.5초
  emotional_appeal  : 평균 정답률 70.0% | 평균 시간 131.8초
  expert_role       : 평균 정답률 70.0% | 평균 시간 155.8초

🤖 모델

결과 저장 및 내보내기

In [15]:
# 결과 저장
print("💾 결과 저장")
print("=" * 50)

solver.export_detailed_results()

# 저장된 파일 확인
import os
if os.path.exists("data/results"):
    print(f"\n📁 저장된 파일 목록:")
    for file in os.listdir("data/results"):
        file_path = os.path.join("data/results", file)
        file_size = os.path.getsize(file_path)
        print(f"   - {file} ({file_size} bytes)")

💾 결과 저장

💾 종합 결과 저장 완료:
   - JSON 요약: data/results/comprehensive_summary.json
   - CSV 요약: data/results/comprehensive_summary.csv
   - 개별 상세 결과: data/results/ 폴더의 각 CSV 파일들

📁 저장된 파일 목록:
   - expert_role_gpt-4o.csv (5479 bytes)
   - emotional_appeal_gpt-4o-mini.csv (4921 bytes)
   - emotional_appeal_gpt-4o.csv (4533 bytes)
   - zero_shot_gpt-4o-mini.csv (797 bytes)
   - expert_role_gpt-4o-mini.csv (5389 bytes)
   - comprehensive_summary.csv (150 bytes)
   - comprehensive_summary.json (220 bytes)
   - zero_shot_gpt-4o.csv (1573 bytes)


## 🧪 전체 테스트 실행

In [17]:
# 전체 테스트 실행

print("🔥 전체 테스트 실행 (모든 문제)")
print("⚠️  경고: 시간과 API 비용이 많이 소요됩니다!")
print("=" * 60)

# 전체 모델로 테스트
full_models = ["gpt-4o-mini", "gpt-4o"]

# 전체 테스트 실행
solver.run_comprehensive_test(
    models=full_models, 
    max_questions=None  # 전체 문제
)

# 최종 결과 분석
solver.analyze_results()
solver.export_detailed_results()


print("전체 테스트 셀이 준비되었습니다.")
print("필요시 위의 주석을 해제하고 실행하세요.")

🔥 전체 테스트 실행 (모든 문제)
⚠️  경고: 시간과 API 비용이 많이 소요됩니다!
🎯 2023 수능 국어 AI 풀이 프로그램 (종합 테스트)
📊 테스트할 문제 수: 전체
🤖 모델: gpt-4o-mini, gpt-4o
💡 프롬프트 기법: zero_shot, emotional_appeal, expert_role
🔥 총 테스트 조합: 2 × 3 = 6개

🚀 테스트 조합 1/6: gpt-4o-mini + zero_shot

🚀 테스트 시작: gpt-4o-mini + zero_shot
⏰ 시작 시간: 2025-05-25 17:42:25

🔍 진행률: 1/11 지문

📖 지문 ID: 2023_11_KICE_1-3
   지문 길이: 978자, 문제 수: 3
--------------------------------------------------
  ✅ 문제 1: 예측=4, 정답=4, 점수=2/2
     응답: 정답: 4
     처리시간: 1.19초
  ❌ 문제 2: 예측=4, 정답=5, 점수=0/3
     응답: 정답: 4번
     처리시간: 1.00초
  ✅ 문제 3: 예측=1, 정답=1, 점수=2/2
     응답: 정답: 1번
     처리시간: 6.91초
   지문 완료 - 현재 총점: 4점

🔍 진행률: 2/11 지문

📖 지문 ID: 2023_11_KICE_4-9
   지문 길이: 2158자, 문제 수: 6
--------------------------------------------------
  ✅ 문제 1: 예측=4, 정답=4, 점수=2/2
     응답: 정답: 4번
     처리시간: 0.78초
  ✅ 문제 2: 예측=5, 정답=5, 점수=2/2
     응답: 정답: 5
     처리시간: 1.61초
  ✅ 문제 3: 예측=3, 정답=3, 점수=2/2
     응답: 정답: 3번
     처리시간: 0.87초
  ✅ 문제 4: 예측=2, 정답=2, 점수=2/2
     응답: 정답: 2번
     처리시간: 0.95초
  ✅ 문제 

In [21]:
# 개별 문제로 테스트해보기 (디버깅 및 확인용)
print("🔍 개별 문제 테스트")
print("=" * 50)

if solver.grouped_data:
    # 첫 번째 지문의 첫 번째 문제로 테스트
    first_paragraph = solver.grouped_data[0]
    first_problem = first_paragraph['problems'][0]
    
    print(f"📖 테스트 지문 ID: {first_paragraph['id']}")
    print(f"📝 테스트 문제:")
    print(f"   {first_problem['question'][:200]}...")
    
    # 프롬프트 생성 테스트
    for prompt_name in ["zero_shot", "emotional_appeal", "expert_role"]:
        print(f"\n💡 {prompt_name} 프롬프트:")
        try:
            prompt = solver.build_prompt_for_problem(
                prompt_name, 
                first_paragraph['paragraph'], 
                first_problem
            )
            print(f"   프롬프트 길이: {len(prompt)}자")
            print(f"   프롬프트 미리보기: {prompt[:300]}...")
        except Exception as e:
            print(f"   ❌ 프롬프트 생성 실패: {e}")

🔍 개별 문제 테스트
📖 테스트 지문 ID: 2023_11_KICE_1-3
📝 테스트 문제:
   윗글의 내용과 일치하지 않는 것은?...

💡 zero_shot 프롬프트:
   프롬프트 길이: 1356자
   프롬프트 미리보기: 다음 지문을 읽고 문제를 풀어주세요.

    지문:
    사람들이 지속적으로 책을 읽는 이유 중 하나는 즐거움이다. 독서의 즐거움에는 여러 가지가 있겠지만 그 중심에는 ‘소통의 즐거움’이 있다.독자는 독서를 통해 책과 소통하는 즐거움을 경험한다. 독서는필자와 간접적으로 대화하는 소통 행위이다. 독자는 자신이 속한사회나 시대의 영향 아래 필자가 속해 있거나 드러내고자 하는 사회나 시대를 경험한다. 직접 경험하지 못했던 다양한 삶을 필자를 매개로 만나고 이해하면서 독자는 더 넓은 시야로 세계를바라볼 수 있다. 이때 같은 책을 읽은 ...

💡 emotional_appeal 프롬프트:
   프롬프트 길이: 1671자
   프롬프트 미리보기: 🎯 인생에서 최고로 중요한 수능 국어 문제입니다.
    당신이 소중하게 생각하는 할머니께서 당신이 좋은 성적을 얻기만을 기다리고 계십니다. 당신의 실력을 보여주세요.
    이 문제는 많은 수험생들이 어려워하는 유형입니다. 하지만 당신은 분명히 풀 수 있습니다! 
    차근차근 읽고, 단계별로 생각해서 최선의 답을 찾아주세요.
    자신감을 가지고 열정적으로 풀어보세요.

    지문:
    사람들이 지속적으로 책을 읽는 이유 중 하나는 즐거움이다. 독서의 즐거움에는 여러 가지가 있겠지만 그 중심에는 ‘소통의 즐거움’이 있다...

💡 expert_role 프롬프트:
   프롬프트 길이: 1364자
   프롬프트 미리보기: 
    당신은 국어교육 전문가입니다. 다음 지문과 문제를 분석하여 정확한 답을 제시해주세요.

    지문:
    사람들이 지속적으로 책을 읽는 이유 중 하나는 즐거움이다. 독서의 즐거움에는 여러 가지가 있겠지만 그 중심에는 ‘소통의 즐거움’이 있다.독자는 독서를 통해

In [22]:
# 저장된 결과 확인
print("📊 저장된 결과 확인")
print("=" * 50)

if solver.results:
    for name, result in solver.results.items():
        print(f"\n📈 {name} 결과:")
        print(f"   정답률: {result.score:.1f}%")
        print(f"   총점: {result.total_score}점")
        print(f"   실행시간: {result.execution_time:.2f}초")
        
        # DataFrame 미리보기
        df = result.detailed_results
        print(f"   데이터프레임 형태: {df.shape}")
        print(f"   컬럼: {list(df.columns)}")
        
        # 처음 3개 결과 미리보기
        print(f"\n   📋 처음 3개 결과:")
        for i in range(min(3, len(df))):
            row = df.iloc[i]
            status = "✅" if row['is_correct'] else "❌"
            print(f"      {status} 문제 {i+1}: 예측={row['pred']}, 정답={row['answer']}, 점수={row['score']}")
else:
    print("저장된 결과가 없습니다. 먼저 테스트를 실행하세요.")

📊 저장된 결과 확인

📈 zero_shot_gpt-4o-mini 결과:
   정답률: 71.1%
   총점: 71점
   실행시간: 97.71초
   데이터프레임 형태: (45, 10)
   컬럼: ['paragraph_id', 'problem_num', 'question', 'pred', 'answer', 'score', 'max_score', 'is_correct', 'response', 'processing_time']

   📋 처음 3개 결과:
      ✅ 문제 1: 예측=4, 정답=4, 점수=2
      ❌ 문제 2: 예측=4, 정답=5, 점수=0
      ✅ 문제 3: 예측=1, 정답=1, 점수=2

📈 emotional_appeal_gpt-4o-mini 결과:
   정답률: 55.6%
   총점: 56점
   실행시간: 140.15초
   데이터프레임 형태: (45, 10)
   컬럼: ['paragraph_id', 'problem_num', 'question', 'pred', 'answer', 'score', 'max_score', 'is_correct', 'response', 'processing_time']

   📋 처음 3개 결과:
      ❌ 문제 1: 예측=2, 정답=4, 점수=0
      ✅ 문제 2: 예측=5, 정답=5, 점수=3
      ✅ 문제 3: 예측=1, 정답=1, 점수=2

📈 expert_role_gpt-4o-mini 결과:
   정답률: 57.8%
   총점: 58점
   실행시간: 189.41초
   데이터프레임 형태: (45, 10)
   컬럼: ['paragraph_id', 'problem_num', 'question', 'pred', 'answer', 'score', 'max_score', 'is_correct', 'response', 'processing_time']

   📋 처음 3개 결과:
      ✅ 문제 1: 예측=4, 정답=4, 점수=2
      ❌ 문제 2: 예측=4, 정답=5,

## 🧪 프롬프트 비교 분석

In [23]:
# 프롬프트별 성능 비교 분석
def compare_prompts(solver, model="gpt-4o-mini", num_questions=5):
    """동일한 모델로 다른 프롬프트들의 성능 비교"""
    print(f"🔬 프롬프트 비교 분석: {model} ({num_questions}문제)")
    print("=" * 60)
    
    prompts = ["zero_shot", "emotional_appeal", "expert_role"]
    results = {}
    
    for prompt in prompts:
        print(f"\n🧪 {prompt} 테스트 중...")
        try:
            result = solver.run_test(prompt, model, num_questions)
            results[prompt] = result
            print(f"   ✅ {prompt}: {result.score:.1f}% 정답률")
        except Exception as e:
            print(f"   ❌ {prompt}: 테스트 실패 - {e}")
    
    # 결과 비교
    if results:
        print(f"\n📊 프롬프트 비교 결과:")
        print("-" * 40)
        sorted_results = sorted(results.items(), key=lambda x: x[1].score, reverse=True)
        
        for i, (prompt, result) in enumerate(sorted_results):
            rank = "🥇" if i == 0 else "🥈" if i == 1 else "🥉"
            print(f"{rank} {prompt:<18}: {result.score:.1f}% ({result.total_score}점)")
    
    return results

print("🔬 프롬프트 비교 분석 함수가 정의되었습니다.")
print("사용법: compare_prompts(solver, 'gpt-4o-mini', 5)")

# 예시 실행 (주석 해제하여 사용하기!)
# prompt_comparison = compare_prompts(solver, 'gpt-4o-mini', 5)

🔬 프롬프트 비교 분석 함수가 정의되었습니다.
사용법: compare_prompts(solver, 'gpt-4o-mini', 5)
